# Script for FIB shaping and ion knife usage in practice

In [1]:
from autoscript_sdb_microscope_client import SdbMicroscopeClient
from autoscript_sdb_microscope_client.structures import *
import time
import os
microscope=SdbMicroscopeClient()
microscope.connect()

Client connecting to [192.168.0.1:7520]...
Client connected to [192.168.0.1:7520]


# 1. Predefined values for gaussian probe and ion knife mode

### Use the following cells if you want to use the pre-defined values for gaussian probe and ion knife. Those will be rough estimates and depend on the system but should correspond to a good starting point in optimization the parameters. In order to optimize these parameters, go to section X. 

In [3]:
imgfocus=0.01859680314795077
imgstig=Point(x=-0.0015345108,y=0.0021776181)
print(imgfocus)
print(imgstig)

millfocus=0.019651918698068503
millstig=Point(x=2.9453514,y=-3)
print(millfocus)
print(millstig)

0.01859680314795077
Point(x=-0.0015345108,y=0.0021776181)


# 2. Switching between modes

## 2.1 Imaging mode

By running the cell below, you will switch to the pre-defined or manually refined parameters for imaging mode.

In [ ]:
microscope.beams.ion_beam.working_distance.value=imgfocus
microscope.beams.ion_beam.stigmator.value=imgstig

## 2.2 Ion knife mode

By running the cell below, you will switch to the pre-defined or manually refined parameters for ion knife milling. In order to use the probe in a spot burn, press ctrl+K in the xT user interface or go to 

In [ ]:
microscope.beams.ion_beam.working_distance.value=millfocus
microscope.beams.ion_beam.stigmator.value=millstig

# 3. Manual refinement of probe parameters
Here you can set refined parameters to be used in switching from gaussian to ion knife beam profile in Section X.


## 3.1 Refine imaging mode
First, adjust the stigmator for normal imaging as you are used to. By running the cell below, you will update the focus and stigmation parameter for the "imaging" or gaussian probe mode

In [10]:
imgfocus=microscope.beams.ion_beam.working_distance.value
imgstig=microscope.beams.ion_beam.stigmator.value
print(imgfocus)
print(imgstig)

0.018995135523610297
Point(x=0.13545046,y=0.0014638018)


## 3.2 Refine ion knife mode
In order to optimize the below, you will update the focus and stigmation parameter for the "ion knife mode" or gaussian probe mode. To do that, perform a gaussian spot burn in imaging mode and subsequently go to ion knife mode. If you start scanning now, you will get the convoluted image of the point-like source of the spot burn with the shape of your ion beam. This crude way of measuring beamshapes represents the basis of beamshape tableau as described with the accompanying paper. Adjust the stigmator and focus parameters as needed to generate the beam shape you wnat to achieve and update the parameters using the cell below. 

In [18]:
millfocus=microscope.beams.ion_beam.working_distance.value
millstig=microscope.beams.ion_beam.stigmator.value
print(millfocus)
print(millstig)

0.020028650478568024
Point(x=3,y=-2.7717362)


# 3.3 Note on milling with the ion knife
If you plan to start an experiment with ion knifes for FIB milling, you will need to measure the beam shift between imaging and ion knife mode. You can do this by milling a spotburn of a gaussian probe and ion knife at the same x-y position in the user interface and subsequently taking and image in imaging mode. The shift between the two spot burns will be the shift you will need to apply to any patterns placed on imaging mode exposures. During this session, the measured shift was 1.4 um upwards in y in scan rotation 0 degree but this will be dependent on your imaging to milling alignments.

# 4. Generating a current dependent spot burn array to test parameters

Enter the desired parameters to be tested in an array spot burn into the list below. This is usually done on a fresh silicon wafer.


In [20]:
currents=[1.5e-12,10e-12,30e-12,50e-12,100e-12,300e-12,500e-12,1e-09,3e-09,5e-09,7e-09,15e-09,30e-09,50e-09,65e-09]
spot_burn_x=5
spot_burn_y=3
milling_time=10
microscope.beams.ion_beam.horizontal_field_width.value

p = StagePosition(x=0.00012949999999999995, y=0)
#microscope.specimen.stage.relative_move(p)

output_dir='D:/SharedData/Sven/20230801_Beamshaping/'

The cell below will generate an array to be exposed, set the parameters for the imaging and ion probe mode and then loop through the different positions to expose with the shaped probe. 

In [13]:
try:
    os.mkdir(output_dir)
except:
    print("Folder already exists!")


def makeSpotArray(x,y):
    '''
    Get Number and return list with different spot burn numbers
    '''
    
    result=[]
    for i in range(x):
        for j in range(y):
            result.append((1.0*(i+0.5)/x,1.0*(j+0.5)/y))
    return(result)


spots=makeSpotArray(spot_burn_x,spot_burn_y)    


slicingset=millstig
slicingfocus=millfocus
preset=imgstig
prefocus=imgfocus

[(0.1, 0.16666666666666666), (0.1, 0.5), (0.1, 0.8333333333333334), (0.3, 0.16666666666666666), (0.3, 0.5), (0.3, 0.8333333333333334), (0.5, 0.16666666666666666), (0.5, 0.5), (0.5, 0.8333333333333334), (0.7, 0.16666666666666666), (0.7, 0.5), (0.7, 0.8333333333333334), (0.9, 0.16666666666666666), (0.9, 0.5), (0.9, 0.8333333333333334)]


In [21]:
stagepos=[]
for current in currents:
    counter=0
    microscope.beams.ion_beam.beam_current.value=current
    microscope.auto_functions.run_auto_cb()
    img=microscope.imaging.grab_frame()
    img.save(output_dir+str(current)+'_beforeExposure.tiff')
    for spot in spots:
        microscope.beams.ion_beam.scanning.mode.set_spot(spot[0],spot[1])
        if counter%3==2:
            microscope.beams.ion_beam.stigmator.value=Point(-slicingset[0],-slicingset[1])
            microscope.beams.ion_beam.working_distance.value=slicingfocus
        elif counter%3==1:
            microscope.beams.ion_beam.stigmator.value=preset
            microscope.beams.ion_beam.working_distance.value=prefocus
        else:
            microscope.beams.ion_beam.stigmator.value=slicingset
            microscope.beams.ion_beam.working_distance.value=slicingfocus
        microscope.beams.ion_beam.unblank()
        time.sleep(milling_time)
        microscope.beams.ion_beam.blank()
        counter=counter+1
    
    microscope.beams.ion_beam.stigmator.value=preset
    microscope.beams.ion_beam.working_distance.value=prefocus
    img=microscope.imaging.grab_frame()
    img.save(output_dir+str(current)+'_afterExposure.tiff')
    stageposition=microscope.specimen.stage.current_position
    stagepos.append(stageposition)
    microscope.specimen.stage.relative_move(p)


microscope.specimen.stage.absolute_move(stagepos[0])

This concludes the introudction to ion knife usage in practice.